In [1]:
import keet_database as kdb
import mediapipe as mp
import cv2
import time
import pandas as pd
import tensorflow as tf
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
##########* LOAD THE MODEL ################################
path_save = 'C:\\Users\\hgera\\OneDrive\\Escritorio\\PY4E\\Keet\\Codes\\NN_FACES\\face_model.h5'
model = tf.keras.models.load_model(path_save)
scaler = joblib.load('C:\\Users\\hgera\\OneDrive\\Escritorio\\PY4E\\Keet\\Codes\\NN_FACES\\scaler_faces.pkl')
dict_labels = {0: 'ENOJO', 1:'FELIZ', 2:'NEUTRAL', 3:'SORPRESA', 4:'TRISTE'}
start_time = time.time()
delay_time = 0.5
predicted = False
prediction = ''
# Open the webcam
cap, width, height = kdb.camera_settings(width_cam= 1280, height_cam= 720, camera=1)

cTime, pTime, fps, Ts, time_frames = kdb.frame_settings()

start = time.time()
i = 0
mp_face_mesh = mp.solutions.face_mesh

face_mesh_images = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=2,
                                         min_detection_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

mp_drawing_styles = mp.solutions.drawing_styles
print('comenzando modelo')
while True:
    
    current_time = time.time()
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Convert grayscale frame to RGB format
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_mesh_results = face_mesh_images.process(rgb_frame)
    
    if face_mesh_results.multi_face_landmarks:
        
        lm = face_mesh_results.multi_face_landmarks
        # Suponiendo que 'lm' es un objeto de tipo NormalizedLandmarkList
        landmarks = lm[0].landmark
        # Extraer las posiciones de los landmarks
        positions_x = np.array([landmark.x for landmark in landmarks])
        positions_y = np.array([landmark.y for landmark in landmarks])
        positions_z = np.array([landmark.z for landmark in landmarks])
        # Dibujar los landmarks en el frame
        # mp_drawing.draw_landmarks(frame, lm[0], connections=mp_face_mesh.FACEMESH_TESSELATION,
                                #   landmark_drawing_spec=None, 
                                #   connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
        min_x, min_y = np.min(positions_x), np.min(positions_y)
        max_x, max_y = np.max(positions_x), np.max(positions_y)
        # Dibujar un rectángulo alrededor de la cara
        cv2.rectangle(frame, (int(min_x*width), int(min_y*height)), 
                      (int(max_x*width), int(max_y*height)), (255, 0, 0), 2)
        flag = 1
    else:
        flag = 0
    # Detect faces in the frame

    #fps = kdb.calculate_fps()
    # Mostrar FPS en el fotograma
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime 
    Ts = time.time() - time_frames
    time_frames = time.time()
    cv2.putText(frame, f"FPS: {int(fps)}", (20, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    
    
    if current_time - start_time >= delay_time:  
        if flag == 1:
            data =  [ np.reshape(positions_x,(468,1))
                     , np.reshape(positions_y,(468,1))]      
            data = np.concatenate(data,axis=1)
            data = data.reshape(1, 936)
            data_normalized = scaler.transform(data)
            predictions = model.predict(data_normalized, verbose=1)
            predicted_class = np.argmax(predictions, axis=1)
            print(f'Predicción: {dict_labels[predicted_class[0]]}') 
            prediction = dict_labels[predicted_class[0]]
            predicted = True
        start_time = current_time      
    if prediction:
        cv2.putText(frame,prediction, (int(max_x), int(min_y)-40), cv2.FONT_HERSHEY_SIMPLEX, 2, (50, 50, 200), 3)
    
    
    
    
    
    
    
    # Mostrar el frame resultante
    cv2.imshow('Face Detection', frame)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()


Resolution: 1280,720
using external camera
comenzando modelo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicción: SORPRESA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicción: SORP

In [3]:
##########* LOAD THE MODEL ################################
import time
import cv2
import mediapipe as mp
import numpy as np
import joblib
import tensorflow as tf

# Cargar modelo y escalador
path_save_faces = 'C:\\Users\\hgera\\OneDrive\\Escritorio\\PY4E\\Keet\\Codes\\NN_FACES\\face_model.h5'
model_faces = tf.keras.models.load_model(path_save)
scaler_faces = joblib.load('C:\\Users\\hgera\\OneDrive\\Escritorio\\PY4E\\Keet\\Codes\\NN_FACES\\scaler_faces.pkl')
dict_labels_faces = {0: 'ENOJO', 1: 'FELIZ', 2: 'NEUTRAL', 3: 'SORPRESA', 4: 'TRISTE'}

# Configuración de la cámara
cap = cv2.VideoCapture(0)
width, height = 1280, 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Variables de tiempo y predicción
start_time = time.time()
delay_time = 0.5
predicted = False
prediction = ""

# Variables de FPS
pTime = 0

# Mediapipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh_images = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Loop principal
while True:
    current_time = time.time()
    # Capturar frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convertir a RGB para Mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_mesh_results = face_mesh_images.process(rgb_frame)
    
    if face_mesh_results.multi_face_landmarks:
        lm = face_mesh_results.multi_face_landmarks[0]  # Solo usar la primera cara detectada
        landmarks = lm.landmark
        # Extraer las posiciones de los landmarks
        positions_x = np.array([landmark.x for landmark in landmarks])
        positions_y = np.array([landmark.y for landmark in landmarks])

        # Calcular el rectángulo alrededor de la cara
        min_x, min_y = np.min(positions_x), np.min(positions_y)
        max_x, max_y = np.max(positions_x), np.max(positions_y)

        # Dibujar el rectángulo alrededor de la cara
        cv2.rectangle(frame, (int(min_x * width), int(min_y * height)),
                      (int(max_x * width), int(max_y * height)), (255, 0, 0), 2)
        
        flag_face = 1
    else:
        flag_face = 0

    # Calcular FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(frame, f"FPS: {int(fps)}", (20, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

    # Hacer la predicción cada cierto tiempo
    if current_time - start_time >= delay_time:  
        if flag == 1:
            data = np.concatenate([
                np.reshape(positions_x, (468, 1)),
                np.reshape(positions_y, (468, 1))
            ], axis=1)
            data = data.reshape(1, 936)
            data_normalized = scaler.transform(data)
            predictions = model.predict(data_normalized, verbose=0)
            predicted_class = np.argmax(predictions, axis=1)
            print(f'Predicción: {dict_labels[predicted_class[0]]}')
            prediction = dict_labels[predicted_class[0]]
            predicted = True  # Se realizó una predicción
        start_time = current_time

    # Mostrar la predicción si se ha realizado
    if predicted:
        cv2.putText(frame, prediction, (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (50, 50, 200), 3)

    # Mostrar el frame resultante
    cv2.imshow('Face Detection', frame)

    # Salir si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


NameError: name 'path_save' is not defined